## Supplement 6: Decision Trees and Random Forest

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor, plot_tree,export_graphviz


### 6.2 Programming Task: Regression Trees
The datasets in files train-reg-tree.csv and test-reg-tree.csv contain samples from a synthetic dataset for training a Regression Tree. The dataset consists of 3 columns: the first two
columns, denoted as x1 and x2, represent the input features for each data sample, and the last
column represents target value denoted by y. There are 200 samples in the train-reg-tree.csv
and 100 samples in the test-reg-tree.csv

In [ ]:
# TODO-1 Read data

train =  pd.read_csv("train-songs.csv")
# test = ...


Given a node M containing N M samples, the node impurity can be expressed by the node
sample variance:
$$V_M = \frac{1}{N_M}\sum_{i=1}^{N_M}(Y_i - \overline{Y}_{\!M})^2,$$
where $Y_i$ is the target of the sample $i$ and $Y_M$ is the mean target of all samples in the node. Similar
to equation 6.2, the goodness-of-split is given by:
$$\Delta V =V_P - \biggl(\frac{N_L}{N_P}\cdot V_L + \frac{N_R}{N_P}\cdot V_R\biggl)$$
where $V_P$ , $V_L$ , $V_R$ are the variances of the parent node, left node and right node respectively.

   i\. Complete the missing code in the implementation of the Regression tree.




In [ ]:
# Helper functions
def split(parent_region, feature_index, threshold):
    # Splits data into left and right using threshold condition

    region_left = parent_region[parent_region[:,feature_index] <=threshold]
    region_right = parent_region[parent_region[:,feature_index] >threshold]
    return region_left, region_right




def variance(dataset_y):
    # dataset_y is a numpy array from the dataset containing only target(y) column
    
    #TODO-2 Implement variance


    #return var


def goodness_of_fit(parent_y, l_child_y, r_child_y):
    # All three inputs are numpy arrays from the dataset containing only the target(y) column
    
    # TODO-3 Implement goodness-of-fit



    #return reduction



def find_split_function(dataset):
    # Dataset -> numpy array with all columns of train including target(y) column

    # TODO-4 Find the best split function by looping through all possible split functions and selecting the split function with the maximum goodness_of_fit
    












                    
    # return the best split function as dictionary containing: feature_index, threshold,dataset_left, dataset_right,variance, goodness_of_fit

    #return best_split




In [ ]:
class Node:
    
    def __init__(self, feature_index, threshold, left, right,variance):
        self.left = left
        self.right = right

        self.threshold = threshold
        self.feature_index = feature_index

        self.variance = variance
        
        self.value = None # Non-leaf nodes don't store values.

    def info_string(self):
        '''Generates string containing information about node. Used for tree visualization'''
        split_function= f"X[{self.feature_index}]<= {self.threshold:.3f}"
        mse = f"var = {self.variance:.3f}"
        info_string = f"{split_function}\n{mse}"
        return info_string
        


class LeafNode:
    def __init__(self,dataset_y):
        
        # For regression the leaf node stores the mean of all samples at that node as its value.
        self.value = np.mean(dataset_y)
    
    def info_string(self):
        '''Generates string containing information about node. Used for tree visualization'''
        info_string = f"value = {self.value:.3f}"
        return info_string



In [ ]:
class RegressionTree():
    def __init__(self, min_samples_split=2, max_depth=2): 
             
        # initialize the root of the tree 
        self.root = None
        
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, depth=1):

        # This function will be called recursively to build subsequent deeper nodes. 
        # The function returns a node.
        

        num_samples = dataset.shape[0]
        # Check stopping conditions
        if num_samples>=self.min_samples_split and depth<=self.max_depth:
            # find best split function
            best_split = find_split_function(dataset)

            # check if goodness is positive, else move to leaf node directly
            if best_split["goodness_of_fit"]>0:
                # Build left tree
                left_subtree = self.build_tree(best_split["dataset_left"], depth+1)
                # Build right tree
                right_subtree = self.build_tree(best_split["dataset_right"], depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"],left_subtree, right_subtree, best_split['variance'])
        

        # return leaf node
        return LeafNode(dataset_y=dataset[:,-1])
    

            
    
    def train(self, dataset):
        self.root = self.build_tree(dataset)
        
    def node_predict(self, x_sample, node):
        
        # TODO-5 Return the value if node is leaf, else check the threshold condition 
        # and call the same function (node_predict) with left or right node 
        # depending on the threshold condition. This is also a recursive function like build_tree.





    
    def predict(self, X):

        
        predictions = [self.node_predict(x_sample, self.root) for x_sample in X]
        return predictions

   ii\. Train the above regression tree using the train dataset and obtain the mean square error (MSE) for the model predictions on the test dataset.

In [ ]:
# Note: Run this cell without any changes. The model will train if the implementation of subtask (i) is correct.
# Make sure you complete all 5 TODO for above.

regressor = RegressionTree(min_samples_split=3,max_depth=4)
regressor.train(train)
y = regressor.predict(test[:,:2])

mse_our = np.mean((test[:,2] -y)**2)
print('Our regression-tree MSE:',mse_our)

   iii\. Compare your results using the model trained using the DecisionTreeRegressor class from the scikit-learn library.

In [ ]:
# TODO-6 Train and predict using scikit-learn library
# clf = DecisionTreeRegressor(min_samples_split=3, max_depth=4)



### OPTIONAL: Tree Visualization
Please install graphviz library.
If you are using anaconda, run:

conda install -c conda-forge python-graphviz


In [ ]:
from graphviz import Digraph, Source


def print_tree(current_node,graph,parent_node_name='root'):
    ''' function to print the tree '''
    graph.node(parent_node_name,current_node.info_string())
    
    if current_node.value is None:        
    
        left_node_name = parent_node_name+'L'
        right_node_name = parent_node_name+'R'

        graph = print_tree(current_node.left, graph, left_node_name)
        graph = print_tree(current_node.right, graph, right_node_name)

        graph.edge(parent_node_name,left_node_name,len='1.00')
        graph.edge(parent_node_name,right_node_name,len='1.00')

    return graph



In [ ]:
reg_tree_graph = Digraph(filename='reg-tree')
reg_tree_graph.attr('node', shape='box')
reg_tree_graph.attr('edge',fixedsize='true')

print_tree(regressor.root, reg_tree_graph).view()

In [ ]:
dot_data = export_graphviz(clf, out_file=None,rounded=True, filled=True)  

graph = Source(dot_data,filename='sklearn-reg-tree')
graph.view()